## TA's Solution

In [48]:
#!/usr/bin/env python

 # float-point division

import sys
from collections import defaultdict, Counter
import itertools
import numpy as np
import time

def process_data(filename):
    X, Y = [], []
    for j, line in enumerate(open(filename)):
        line = line.strip()
        features = line.split(", ")
        feat_vec = np.zeros(dimension)
        for i, fv in enumerate(features[:-1]): # last one is target
            if i in [0,7]:
                feat_vec[feature_map[i, 0]] = float(fv) / 50.  # NB: diff 2 not 1!
            elif (i, fv) in feature_map: # ignore unobserved features
                feat_vec[feature_map[i, fv]] = 1

        X.append(feat_vec)
        Y.append(1 if features[-1] == ">50K" else -1) # fake for testdata

    return np.array(X), np.array(Y)


def process_mx_dist(testX, trainX):
    mx_dist=[]
    for example in testX:
        mx_dist.append(np.linalg.norm(example - trainX, axis=1))
    return np.argsort(np.array(mx_dist),axis=1)

def knn(k, trainY, mx_dist):
    return np.sum(trainY[mx_dist[:,:k]],axis=1)

def eval(k, test_XY, trainY, mx_dist):
    (testX, testY) = test_XY
    pred=np.array([1 if x>0 else -1 for x in knn(k, trainY,mx_dist)])
    errors = sum(pred != testY)
    positives = sum(pred == 1)
    return errors * 100. / len(testX) , positives * 100. / len(testX) 

if __name__ == "__main__":
    field_value_freqs = defaultdict(lambda : defaultdict(int)) # field_id -> value -> freq
    for line in open("income.train.txt.5k"):
        line = line.strip()
        features = line.split(", ")[:-1] # exclude target label
        for i, fv in enumerate(features):
            field_value_freqs[i][0 if i in [0,7] else fv] += 1

    feature_map = {}
    feature_remap = {}
    for i, value_freqs in field_value_freqs.items():
        for v in value_freqs:
            k = len(feature_map) # bias
            feature_map[i, v] = k
            feature_remap[k] = i, v

    dimension = len(feature_map) # bias
    print("dimensionality: ", dimension) #, feature_map

    train_data = process_data("income.train.txt.5k") 
    dev_data = process_data("income.dev.txt")
    #test_data = process_data("income.test.txt")
    
    mx_dist_train=process_mx_dist(train_data[0][:5000],train_data[0][:5000])
#     mx_dist_dev=process_mx_dist(dev_data[0][:1000],train_data[0][:5000])
    print(len(mx_dist_train[1]))
    print(mx_dist_train[:2])

    kVals = [1,3,5,7,9,41,99,999,1999,2999,3999,4999,9999]
    for k in kVals:
#     for k in map(int, sys.argv[1:]):     
        print("k=%-4d" % k,
              'train_err %4.1f%% (+:%5.1f%%)' % eval(k, (train_data[0][:5000], train_data[1][:5000]), train_data[1][:5000],mx_dist_train),
              'dev_err %4.1f%% (+:%5.1f%%)' % eval(k, (dev_data[0][:1000], dev_data[1][:1000]), train_data[1][:5000],mx_dist_dev))

dimensionality:  92
5000
[[   0 2089 4828 ... 4465 3516 3975]
 [   1 2533  811 ... 4306 1931 3793]]
k=1    train_err  1.5% (+: 25.2%) dev_err 23.0% (+: 27.6%)
k=3    train_err 11.4% (+: 24.1%) dev_err 19.6% (+: 26.2%)
k=5    train_err 13.8% (+: 24.2%) dev_err 17.4% (+: 24.6%)
k=7    train_err 14.7% (+: 23.9%) dev_err 16.0% (+: 24.0%)
k=9    train_err 15.5% (+: 24.2%) dev_err 15.8% (+: 21.8%)
k=41   train_err 17.5% (+: 20.8%) dev_err 14.8% (+: 20.0%)
k=99   train_err 17.9% (+: 19.5%) dev_err 15.7% (+: 18.9%)
k=999  train_err 20.2% (+: 10.5%) dev_err 18.1% (+: 11.1%)
k=1999 train_err 25.0% (+:  0.0%) dev_err 23.6% (+:  0.0%)
k=2999 train_err 25.0% (+:  0.0%) dev_err 23.6% (+:  0.0%)
k=3999 train_err 25.0% (+:  0.0%) dev_err 23.6% (+:  0.0%)
k=4999 train_err 25.0% (+:  0.0%) dev_err 23.6% (+:  0.0%)
k=9999 train_err 25.0% (+:  0.0%) dev_err 23.6% (+:  0.0%)


In [56]:
from collections import defaultdict

a = defaultdict(lambda : defaultdict(int))
a[0][0] = 2
a[1]['Private'] = 4
a[2]['Bachelors'] = 3
a[1]['Self-Employed'] = 30
a[2]['High School'] = 6

for i,row in a.items():
    print('i: {} row: {}'.format(i,row))
    for v in row:
        print(v)

i: 0 row: defaultdict(<class 'int'>, {0: 2})
0
i: 1 row: defaultdict(<class 'int'>, {'Private': 4, 'Self-Employed': 30})
Private
Self-Employed
i: 2 row: defaultdict(<class 'int'>, {'Bachelors': 3, 'High School': 6})
Bachelors
High School


## Professor's solution (Has a bug)

In [49]:
#!/usr/bin/env python

from __future__ import division # float-point division

import sys
from collections import defaultdict, Counter
import itertools
import numpy as np
import time

def process_data(filename):
    X, Y = [], []
    for j, line in enumerate(open(filename)):
        line = line.strip()
        features = line.split(", ")
        feat_vec = np.zeros(dimension)
        for i, fv in enumerate(features[:-1]): # last one is target
            if i in [0,7]:
                feat_vec[feature_map[i, 0]] = float(fv) / 50  # NB: diff 2 not 1!
            elif (i, fv) in feature_map: # ignore unobserved features
                feat_vec[feature_map[i, fv]] = 1

        X.append(feat_vec)
        Y.append(1 if features[-1] == ">50K" else -1) # fake for testdata

    return np.array(X), np.array(Y)

def knn(k, example, (trainX, trainY)):
    #dists = [(np.linalg.norm(example - vecx), y) for vecx, y in train_data]
    neighbors = np.argpartition(np.linalg.norm(example - trainX, axis=1), k)[:k]
    #neighbors = np.argsort(np.linalg.norm(example - trainX, axis=1))[:k]
    votes = trainY[neighbors] # slicing
    return 1 if sum(votes) > 0 else -1
    #return Counter(votes).most_common(1)[0][0] # [(1, 6), (-1, 4)] 

def eval(k, (testX, testY), train):
    pred = np.array([knn(k, vecx, train) for vecx in testX])
    errors = sum(pred != testY)
    positives = sum(pred == 1)
    return errors / len(testX) * 100, positives / len(testX) * 100

if __name__ == "__main__":
    field_value_freqs = defaultdict(lambda : defaultdict(int)) # field_id -> value -> freq
    for line in open("income.train.txt.5k"):
        line = line.strip()
        features = line.split(", ")[:-1] # exclude target label
        for i, fv in enumerate(features):
            field_value_freqs[i][0 if i in [0,7] else fv] += 1

    feature_map = {}
    feature_remap = {}
    for i, value_freqs in field_value_freqs.iteritems():
        for v in value_freqs:
            k = len(feature_map) # bias
            feature_map[i, v] = k
            feature_remap[k] = i, v

    dimension = len(feature_map) # bias
    print "dimensionality: ", dimension #, feature_map

    train_data = process_data("income.train.txt.5k") 
    dev_data = process_data("income.dev.txt")
    #test_data = process_data("income.test.txt")

    for k in map(int, sys.argv[1:]):
        print "k=%-4d" % k,
        if k > len(train_data):
            k = -1 # infinity; N.B. bug in np.argpartition with very large k
        print 'train_err %4.1f%% (+:%5.1f%%)' % eval(k, (train_data[0][:5000], train_data[1][:5000]), train_data),
        print 'dev_err %4.1f%% (+:%5.1f%%)' % eval(k, (dev_data[0][:1000], dev_data[1][:1000]), train_data)

SyntaxError: invalid syntax (<ipython-input-49-369228746106>, line 28)

In [47]:
import numpy as np
from collections import defaultdict

def getData(fileName):
    with open(fileName) as trainTxt:
        train = trainTxt.readlines()

        #Getting all data except the target column
        data = list(map(lambda s : s.strip().split(', '), train))
    return data

def getAllData(fileName):
    with open(fileName) as trainTxt:
        train = trainTxt.readlines()

    #Getting all data except the target column
    data = list(map(lambda s : s.strip().split(', '), train))
        
    sector = [line[1] for line in data]
    education = [line[2] for line in data]
    marital = [line[3] for line in data]
    occ = [line[4] for line in data]
    race = [line[5] for line in data]
    gender = [line[6] for line in data]
    country = [line[8] for line in data]
    
    age = [x/50 for x in list(map(int,[line[0] for line in data]))]
    hours = [x/50 for x in list(map(int,[line[7] for line in data]))]
    
    return list(zip(age, sector, education, marital, occ, race, gender, country, hours))

def getFeatureMapAndEncodedData(trainData):
    featureMapping = {}
    encodedTrainData = []
    
    for row in trainData:
        newRow = []
        for j,x in enumerate(row):
            feature = (j,x)
            if feature not in featureMapping:
                featureMapping[feature] = len(featureMapping)
            newRow.append(featureMapping[feature])
        encodedTrainData.append(newRow)
        
    return featureMapping, encodedTrainData

def getTranslatedOnFeatureMap(featureMapping, data):
    translatedData = []
    
    for row in data:
        newRow = []
        for j,x in enumerate(row):
            feature = (j,x)
            if j not in [0,8]:
                if feature in featureMapping:
                    newRow.append(featureMapping[feature])
            elif j in [0,8]:
                newRow.append(x)
        translatedData.append(newRow)
    return translatedData

def getBinarizedData(unbinarizedData, numberOfFeatures):
    finalData = np.zeros((len(unbinarizedData), numberOfFeatures))
    for c, row in enumerate(unbinarizedData):
        for i,x in enumerate(row):
            if isinstance(x,int) and i not in [0,8]:
                finalData[c][i] = 1
            elif isinstance(x,float) and i in [0,8]:
                finalData[c][i] = x
    return finalData

def getPredictions(topKElements, fileName):
    actualIncome = getTarget(getData(fileName))
    predictions = []
    
    for i in range(len(topKElements)):
        predictedLabelsIndex = [actualIncome[j] for j in topKElements[i]]
        predictedLabel = Counter(predictedLabelsIndex).most_common(1)[0][0]
        predictions.append(predictedLabel)
    return predictions

def getNormWithoutIndex(train, test, ord=None):
    dist = []
    for row in test:
        dist.append(np.linalg.norm(row - train, ord, axis = 1))
    return dist

def getErrorRate(predictions, actualTargetFileName):
    actuals = getTarget(getData(actualTargetFileName))
    error = 0
    for i in range(len(predictions)):
        if actuals[i] != predictions[i]:
            error+=1
    return error/10

def getPositives(predictions):
    return Counter(predictions)['>50K']/10

def getErrorRatesAndPositivePercentage(distWithoutIndex, baselineFileName):
    kList = [1,3,5,7,9,31,32,33,34,35,40,99,999,9999]
    
    topKElements = defaultdict(list)
    predictions = defaultdict(list)

    for k in kList:
        if len(distWithoutIndex) < k:
            k = len(distWithoutIndex)
            topKElements[k] = [np.argpartition(x,k-1, axis=0)[:k-1] for x in [distWithoutIndex[i] for i in range(len(distWithoutIndex))]]
        else:
            topKElements[k] = [np.argpartition(x,k, axis=0)[:k] for x in [distWithoutIndex[i] for i in range(len(distWithoutIndex))]]
        predictions[k] = getPredictions(topKElements[k], baselineFileName)

    return predictions

def getTarget(data):
    return [line[-1] for line in data]

if __name__ == '__main__':
    featureMapping, unbinarizedTrainData = getFeatureMapAndEncodedData(getAllData('income.train.txt.5k'))
    unbinarizedTrainData = getTranslatedOnFeatureMap(featureMapping, getAllData('income.train.txt.5k'))
#     unbinarizedDevData = getTranslatedOnFeatureMap(featureMapping, getAllData('income.dev.txt'))
#     unbinarizedTestData = getTranslatedOnFeatureMap(featureMapping, getAllData('income.test.blind'))

    print(unbinarizedTrainData[:2])
    binarizedTrainData = getBinarizedData(unbinarizedTrainData, 92)
#     binarizedDevData = getBinarizedData(unbinarizedDevData, 92)
#     binarizedTestData = getBinarizedData(unbinarizedTestData, 92)

    print('------For train data---------')
    trainDistWithoutIndex = getNormWithoutIndex(binarizedTrainData, binarizedTrainData)
    print(trainDistWithoutIndex[:2])
    trainPredictions = getErrorRatesAndPositivePercentage(trainDistWithoutIndex, 'income.train.txt.5k')
    for k,preds in trainPredictions.items():
        print('k: {} error rate: {} positive: {}'.format(k,getErrorRate(preds, 'income.train.txt.5k'), getPositives(preds)))


[[1.0, 1, 2, 3, 4, 5, 6, 7, 0.26], [0.76, 10, 11, 12, 13, 5, 6, 7, 0.8]]
------For train data---------
[array([0.        , 0.59093147, 0.54332311, ..., 0.27202941, 0.56320511,
       0.79246451]), array([0.59093147, 0.        , 0.3       , ..., 0.83761566, 0.08      ,
       0.34      ])]
k: 1 error rate: 124.2 positive: 108.5
k: 3 error rate: 113.6 positive: 79.1
k: 5 error rate: 121.2 positive: 89.5
k: 7 error rate: 114.4 positive: 66.5
k: 9 error rate: 116.7 positive: 73.4
k: 31 error rate: 119.4 positive: 54.9
k: 32 error rate: 120.2 positive: 56.9
k: 33 error rate: 118.5 positive: 53.0
k: 34 error rate: 120.1 positive: 55.0
k: 35 error rate: 118.8 positive: 52.5
k: 40 error rate: 118.1 positive: 43.4
k: 99 error rate: 120.9 positive: 46.4
k: 999 error rate: 125.1 positive: 0.0
k: 5000 error rate: 125.1 positive: 0.0
